In [1]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, dijkstra, format_path

from pprint import pprint
import numpy as np


importing Jupyter notebook from dijkstra.ipynb
('1', '2', '3', '4', '5n', '5s', '6', '7', '8', '9', '10', '10b', '11n', '11s', '12', '13', '14', '15', '16', '17', '19', '20', '20b', '21', '22', '23', '24', '25/1', '25/2', '26', '31', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22')
[[inf inf  7. ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [ 7. inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf  4. inf]
 [inf inf inf ...  4. inf inf]
 [inf inf inf ... inf inf inf]]
X1 -> X2 -> 9 -> 7
9.0
2: [2, 36, 35, 35, 33, 33, 32, 47, 32, 7, 37, 42, 43, 44, 45, 41, 40, 40, 17, 46, 50, 52, 52, -1, 49, 48, 47, 51, 27, 50, 25, 32, 9, 9, 33, 34, 34, 9, 37, 38, 39, 16, 39, 38, 45, 46, 47, 48, 49, 50, 51, 21, 23]
1: [55.0, 55.0, 48.0, 47.0, 45.0, 44.0, 40.0, 35.0, 40.0, 37.0, 42.0, 48.0, 45.0, 39.0, 38.0, 51.0, 47.0, 47.0, 50.0, 35.0, 22.0, 8.0, 3.0, 0, 26.0, 31.0, 32.0, 20.0, 22.0, 22.0, 33.0, 44.0, 39.0

In [2]:
def init_tamponi_per_reparto():
    tamponi_per_reparto = np.array([ 0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  7,
        0,  0,  0, 10,  0,  0,  0,  3,  0, 11,  0,  4,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0]).astype(int)

    return tamponi_per_reparto

In [3]:
tamponi_per_reparto = init_tamponi_per_reparto()
print(tamponi_per_reparto)

# # # indice del "31"
# # index_31 = nodes.index("31")
# # tamponi_per_reparto = np.array([0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 12, 0, 0, 0, 7, 0, 0, 0, 10, 0,
# #         0, 0, 3, 10, 30, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
# #         0, 0, 0, 0, 0, 0, 0, 0, 0]).astype(int)
# # # aggiungi 10 tamponi al reparto 31
# # tamponi_per_reparto[index_31] += 10
# # tamponi_per_reparto

# index_23 = nodes.index("31")
# tamponi_per_reparto = np.array([ 0,  0,  0,  3,  0,  0,  0,  6,  0,  0,  0,  0, 12,  0,  0,  0,  7,
#         0,  0,  0, 10,  0,  0,  0,  3,  0, 30,  0,  4,  0, 10,  0,  0,  0,
#         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
#         0,  0]).astype(int)
# tamponi_per_reparto[index_23] = 0

# pprint(tamponi_per_reparto)

[ 0  0  0  3  0  0  0  0  0  0  0  0 12  0  0  0  7  0  0  0 10  0  0  0
  3  0 11  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [4]:
def estrai_reparti_con_tamponi(tamponi_per_reparto):
    return [i for i,t in enumerate(tamponi_per_reparto) if t > 0]

In [5]:
reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)

print(reparti_con_tamponi)

for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[node]} tamponi")

[3, 12, 16, 20, 24, 26, 28]
0. Reparto con indice 3 e nome '4' ha 3 tamponi
1. Reparto con indice 12 e nome '11n' ha 12 tamponi
2. Reparto con indice 16 e nome '14' ha 7 tamponi
3. Reparto con indice 20 e nome '19' ha 10 tamponi
4. Reparto con indice 24 e nome '22' ha 3 tamponi
5. Reparto con indice 26 e nome '24' ha 11 tamponi
6. Reparto con indice 28 e nome '25/2' ha 4 tamponi


In [6]:
## SCORE MATRIX
MASSIMA_CAPACITA_TRASPORTABILE = 6

# conteggio dei tamponi in ogni nodo dato un path (oppure un nodo di partenza) 
def compute_tamponi_in_nodo(p, prev, tamponi_per_reparto, tamponi_per_path, start_node=None):
    if prev is None and start_node is not None:
        _, prev = np.array(dijkstra(arc_matrix, start_node))
        tamponi_per_path = -1 * np.ones(len(prev))
    
    path = []
    path.append(p)

    prev = np.array(prev).astype(int)
    while p != -1:
        p = prev[p]
        path.append(p)

    path = path[::-1] # reverse
    path = path[1:] # remove start node

    temp_tamponi = 0
    for p in path:
        if tamponi_per_path[p] == -1:
            # print(p,"aggiungo questo reparto al dizionario")
            temp_tamponi += tamponi_per_reparto[p]
            tamponi_per_path[p] = temp_tamponi
        else:
            temp_tamponi = tamponi_per_path[p]
        
        # print(p, nodes[p], tamponi_per_reparto[p], temp_tamponi)
    
    return tamponi_per_path
        

def compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto, tamponi_trasportati, criterio="density", moving_info=(None,None,0)):
    time, prev = np.array(dijkstra(arc_matrix, indice_reparto))
    prev = np.array(prev).astype(int)

    time = time[reparti_con_tamponi]
    
    (moving_from, moving_toward, moving_time) = moving_info
    # moving_from si trova nel grafo ridotto:
    if moving_from is not None: # se hai un'informazione di movimento
        
        for r in reparti_con_tamponi:         
            path,_ = get_shortest_path(arc_matrix, indice_reparto, r, prev)
            # print(indice_reparto, r, path)
            # print(format_path(path))
            r_index = list(reparti_con_tamponi).index(r)
            if moving_toward in path:
                time[r_index] -= moving_time
                # print("il tempo per arrivare da ",nodes[indice_reparto],"a", nodes[r], "è ridotto di", moving_time, "minuti")
            else:
                # print("il tempo per arrivare da ",nodes[indice_reparto],"a", nodes[r], "è aumentato di", moving_time, "minuti")
                time[r_index] += moving_time
    
    if criterio == "density":
        ## TAMPONI PER PATH
        tamponi_per_path = -1 * np.ones(len(prev))
        for reparto in reparti_con_tamponi:
            tamponi_per_path = compute_tamponi_in_nodo(reparto, prev, tamponi_per_reparto, tamponi_per_path)
        ## TAMPONI IN NODO FINALE
        # tamponi_per_path = tamponi_per_reparto.copy()
        tamponi_per_path = tamponi_per_path[reparti_con_tamponi]
        tamponi_per_path = np.minimum(tamponi_per_path, MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati)
        # print(tamponi_per_path)
        
        # print(tamponi_per_path)
        density_score = np.divide(tamponi_per_path, time, out=-np.inf*np.ones_like(time), where=time!=0)
        return density_score
    
    if criterio == "time":
        return time
    
    raise ValueError(f"{criterio} non è un criterio valido. Usa 'density' o 'time'")

# tamponi / minuto
def compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=(None,None,0)):
    return compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)

def compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=(None,None,0)):
    reparti_con_tamponi = np.array(reparti_con_tamponi)
    score_matrix = np.zeros((len(reparti_con_tamponi), len(reparti_con_tamponi)))
    for i, indice_reparto in enumerate(reparti_con_tamponi):
        score_matrix[i] = compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
    return score_matrix

In [7]:
def retrieve_best_score(score_matrix, index=None):
    if index is not None: # per score_matrix
        return np.argmax(score_matrix[index]), np.max(score_matrix[index])
    else:
        return np.argmax(score_matrix), np.max(score_matrix)

In [8]:
# moving_info = (nodes.index("9"), nodes.index("7"), 1)
moving_info = (None, None, 0)
tamponi_trasportati = 0
score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
for index,node in enumerate(reparti_con_tamponi):
    best_index, best_score = retrieve_best_score(score_matrix, index)
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha best score in reparto {best_index}. indice {reparti_con_tamponi[best_index]} con nome '{nodes[reparti_con_tamponi[best_index]]}' con score {best_score}")

0. Reparto con indice 3 e nome '4' ha best score in reparto 5. indice 26 con nome '24' con score 0.35294117647058826
1. Reparto con indice 12 e nome '11n' ha best score in reparto 2. indice 16 con nome '14' con score 0.6
2. Reparto con indice 16 e nome '14' ha best score in reparto 1. indice 12 con nome '11n' con score 0.6
3. Reparto con indice 20 e nome '19' ha best score in reparto 4. indice 24 con nome '22' con score 0.6
4. Reparto con indice 24 e nome '22' ha best score in reparto 3. indice 20 con nome '19' con score 0.6
5. Reparto con indice 26 e nome '24' ha best score in reparto 4. indice 24 con nome '22' con score 0.6
6. Reparto con indice 28 e nome '25/2' ha best score in reparto 3. indice 20 con nome '19' con score 0.42857142857142855


In [9]:
def check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto):
    # controllo: se all'interno del path c'è un nodo con tamponi, allora scelgo quello
    
    path, _ = get_shortest_path(arc_matrix, start_node, reparti_con_tamponi[next_index])
    
    nodo_finale = None
    for p in path[1:-1]:
        if p in reparti_con_tamponi and tamponi_per_reparto[p] > 0:
            nodo_finale = next_index
            next_index = reparti_con_tamponi.index(p)
            break
    
    return next_index, nodo_finale

def get_next_node(start_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto):
    index = reparti_con_tamponi.index(start_node)
    next_index, score = retrieve_best_score(score_matrix, index)
    
    next_index, nodo_finale = check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
    
    score_matrix[:,index] = -np.inf
    return reparti_con_tamponi[next_index], score, nodo_finale

In [10]:
DIMENSIONE_PIASTRA = 96 # tamponi
MASSIMA_CAPACITA_TRASPORTABILE = 6 # tamponi
massimo_ritardo = 0 # minuti
def stop_condition(residual_time, tamponi_trasportati, step, reparti_con_tamponi):
    return residual_time <= -massimo_ritardo or tamponi_trasportati >= MASSIMA_CAPACITA_TRASPORTABILE or step >= len(reparti_con_tamponi)

def multi_stop_condition(residual_time, lab, current_position, tamponi_trasportati, tamponi_per_reparto):
    
    if sum(tamponi_per_reparto) == 0:
        return True
    
    _,lab_return_time = get_shortest_path(arc_matrix, current_position, lab)
    residual_time = residual_time - lab_return_time
    # print(f"M{residual_time <= -massimo_ritardo or tamponi_trasportati >= MASSIMA_CAPACITA_TRASPORTABILE}")
    return residual_time <= -massimo_ritardo or tamponi_trasportati >= MASSIMA_CAPACITA_TRASPORTABILE


In [11]:
def build_tsp_path(arc_matrix, tamponi_per_reparto_orig, residual_time, tamponi_trasportati, current_node, lab, moving_info=(None,None,0)):
    reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto_orig)
    tamponi_per_reparto = tamponi_per_reparto_orig.copy()
    path = []
        
    step = 0
    _,lab_return_time = get_shortest_path(arc_matrix, current_node, lab)
    # se il nodo di partenza non è un nodo con tamponi, allora trovo il nodo con tamponi con score più alto
    if not stop_condition(residual_time - lab_return_time, tamponi_trasportati, step, reparti_con_tamponi) and current_node not in reparti_con_tamponi:
        
        score_row = compute_score_row(arc_matrix, reparti_con_tamponi, current_node, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
        
        ## TODO: controlla se accorpabile con get_next_node
        next_index, score = retrieve_best_score(score_row)
        next_index, nodo_finale = check_reparto_intermedio(current_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
        next_node = reparti_con_tamponi[next_index]
        ## END TODO
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        ## DEBUG:
        # print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
        ##
        
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        current_node = next_node
        moving_info = (None, None, 0)
    
    step += 1
    _,lab_return_time = get_shortest_path(arc_matrix, current_node, lab)
    while not stop_condition(residual_time - lab_return_time, tamponi_trasportati, step, reparti_con_tamponi):
        score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
        next_node, score, nodo_finale = get_next_node(current_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto)
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        ## DEBUG:
        # print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
        ##
        
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        # print(f"Tempo di ritorno al laboratorio da current_node: {lab_time_row[reparti_con_tamponi.index(current_node)]}")
        # print(f"Tempo di ritorno al laboratorio da next_node: {lab_time_row[reparti_con_tamponi.index(next_node)]}")
        current_node = next_node
        step += 1
        # lab_time_row = compute_score_row(arc_matrix, reparti_con_tamponi, lab, tamponi_per_reparto, criterio="time")
        # lab_return_time = lab_time_row[reparti_con_tamponi.index(current_node)]
        _,lab_return_time = get_shortest_path(arc_matrix, current_node, lab)
        moving_info = (None, None, 0)
            
    # path = path[:-1] # remove last node (probabilmente)
    
    path += [lab]
        
    
    return path

In [13]:
# estensione a multi-agente

def multi_build_tsp_path(arc_matrix, env, team_raccolta):
    reparti_con_tamponi = estrai_reparti_con_tamponi(env.tamponi_per_reparto)
    tamponi_per_reparto = env.tamponi_per_reparto.copy()
    
    agent_info = {}
    previous_agent_info = {}
    for addetto in team_raccolta:
        if not addetto in agent_info:
            agent_info[addetto] = {}
            previous_agent_info[addetto] = {}
        
        agent_info[addetto]['residual_time'] = residual_time = env.get_residual_time()
        agent_info[addetto]['current_node'] = current_node = addetto.current_position
        agent_info[addetto]['tamponi_trasportati'] = tamponi_trasportati = addetto.tamponi_trasportati
        agent_info[addetto]['moving_info'] = moving_info = addetto.moving_info
        agent_info[addetto]['path'] = path = []
        agent_info[addetto]['raccolta'] = {}
        
        # print(agent_info[addetto])
    
    maximum_number_of_agents = sum(env.tamponi_per_reparto) // MASSIMA_CAPACITA_TRASPORTABILE
    if maximum_number_of_agents == 0:
        maximum_number_of_agents = 1
    # maximum_number_of_agents = len(team_raccolta)
    
    while agent_info != previous_agent_info:
        for addetto in team_raccolta[:maximum_number_of_agents]:
            
            residual_time = agent_info[addetto]['residual_time']
            current_node = agent_info[addetto]['current_node']
            tamponi_trasportati = agent_info[addetto]['tamponi_trasportati']
            moving_info = agent_info[addetto]['moving_info']
            path = agent_info[addetto]['path']
            
            # se il nodo di partenza non è un nodo con tamponi, allora trovo il nodo con tamponi con score più alto
            if not multi_stop_condition(residual_time, env.lab, current_node, tamponi_trasportati, tamponi_per_reparto):
                if current_node not in reparti_con_tamponi:
                    score_row = compute_score_row(arc_matrix, reparti_con_tamponi, current_node, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
                    
                    ## TODO: controlla se accorpabile con get_next_node
                    next_index, score = retrieve_best_score(score_row)
                    next_index, nodo_finale = check_reparto_intermedio(current_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
                    next_node = reparti_con_tamponi[next_index]
                    ## END TODO
                else:
                    score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, tamponi_trasportati, moving_info=moving_info)
                    next_node, score, nodo_finale = get_next_node(current_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto)
                    
                distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
                tempo_di_percorrenza = distances[next_node]
                residual_time -= tempo_di_percorrenza
                agent_info[addetto]['residual_time'] = residual_time
                tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
                tamponi_raccolti = min(tamponi_per_path[next_node], MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati)
                tamponi_trasportati += tamponi_raccolti
                agent_info[addetto]['tamponi_trasportati'] = tamponi_trasportati
                tamponi_per_reparto[next_node] -= tamponi_raccolti
                agent_info[addetto]['raccolta'][next_node] = tamponi_raccolti
                
                ## DEBUG:
                # print(f"{addetto} '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi per path; tamponi_raccolti: {tamponi_raccolti} / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
                ##
                
                path.append(next_node)
                agent_info[addetto]['path'] = path
                agent_info[addetto]['current_node'] = current_node = next_node
                agent_info[addetto]['moving_info'] = moving_info = (None, None, 0)
            
            reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)
        previous_agent_info = agent_info.copy()
        
    for addetto in team_raccolta[:maximum_number_of_agents]:
        path = agent_info[addetto]['path']
        path += [env.lab]
        agent_info[addetto]['path'] = path
        agent_info[addetto]['current_node'] = addetto.current_position
    
    return agent_info

# for index,node in enumerate(estrai_reparti_con_tamponi(tamponi_per_reparto)):
#     print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[node]} tamponi")

# plan = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, current_node, lab, moving_info=moving_info)
# print(f"{nodes[current_node]} -> {format_path(plan)}")

# agent_info = multi_build_tsp_path(arc_matrix, env, team_raccolta)
# for addetto in team_raccolta:
#     print(f"{addetto}: {nodes[addetto.current_position]} -> {format_path(agent_info[addetto]['path'])}")